In [1]:
import numpy as np
import sys
import os
#import tensorflow as tf

import io
from gensim.models import word2vec

from io import StringIO
from pprint import pprint

from nltk.tokenize import word_tokenize
import nltk

In [2]:
from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import types
from IPython import get_ipython #for import notebook
from nbformat import read #for import notebook
from IPython.core.interactiveshell import InteractiveShell #for import notebook

In [3]:
def find_notebook(fullname, path=None): #for import notebook
    """find a notebook, given its fully qualified name and an optional path

    This turns "foo.bar" into "foo/bar.ipynb"
    and tries turning "Foo_Bar" into "Foo Bar" if Foo_Bar
    does not exist.
    """
    name = fullname.rsplit('.', 1)[-1]
    if not path:
        path = ['']
    for d in path:
        nb_path = os.path.join(d, name + ".ipynb")
        if os.path.isfile(nb_path):
            return nb_path
        # let import Notebook_Name find "Notebook Name.ipynb"
        nb_path = nb_path.replace("_", " ")
        if os.path.isfile(nb_path):
            return nb_path

In [4]:
class NotebookLoader(object): #for import notebook
    """Module Loader for Jupyter Notebooks"""
    def __init__(self, path=None):
        self.shell = InteractiveShell.instance()
        self.path = path

    def load_module(self, fullname):
        """import a notebook as a module"""
        path = find_notebook(fullname, self.path)

        print ("importing Jupyter notebook from %s" % path)

        # load the notebook object
        with io.open(path, 'r', encoding='utf-8') as f:
            nb = read(f, 4)


        # create the module and add it to sys.modules
        # if name in sys.modules:
        #    return sys.modules[name]
        mod = types.ModuleType(fullname)
        mod.__file__ = path
        mod.__loader__ = self
        mod.__dict__['get_ipython'] = get_ipython
        sys.modules[fullname] = mod

        # extra work to ensure that magics that would affect the user_ns
        # actually affect the notebook module's ns
        save_user_ns = self.shell.user_ns
        self.shell.user_ns = mod.__dict__

        try:
          for cell in nb.cells:
            if cell.cell_type == 'code':
                # transform the input to executable Python
                code = self.shell.input_transformer_manager.transform_cell(cell.source)
                # run the code in themodule
                exec(code, mod.__dict__)
        finally:
            self.shell.user_ns = save_user_ns
        return mod
class NotebookFinder(object):
    """Module finder that locates Jupyter Notebooks"""
    def __init__(self):
        self.loaders = {}

    def find_module(self, fullname, path=None):
        nb_path = find_notebook(fullname, path)
        if not nb_path:
            return

        key = path
        if path:
            # lists aren't hashable
            key = os.path.sep.join(path)

        if key not in self.loaders:
            self.loaders[key] = NotebookLoader(path)
        return self.loaders[key]

In [5]:
sys.meta_path.append(NotebookFinder())

In [6]:
import torch
import torch.nn as nn
from torch import optim

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
import torch.utils.data
from torchvision import transforms, utils
from torch.utils.data import DataLoader

In [7]:
import pre_process

importing Jupyter notebook from pre_process.ipynb

 [[1561, 1, 1089, 1, 102, 33, 74, 14, 7, 229, 3510, 218, 354, 4], [32, 39, 16, 10, 4904, 43, 10, 68, 44, 60, 14, 81, 2579, 0], [29, 21, 3, 225, 4, 41, 35, 93, 149, 6, 1314, 0], [71, 3, 68, 38, 40, 4, 2, 64, 0, 41, 35, 51, 106, 149, 1473, 8, 2582, 1735, 0, 1409, 177, 61, 4], [2, 325, 3, 19, 6800, 57, 654, 30, 21, 4, 2, 64, 207, 24, 1492, 36, 224, 0], [2, 746, 7, 547, 144, 6, 5, 1215, 230, 30, 23, 348, 4963, 8, 273, 56, 12, 81, 358, 0], [97, 7, 60, 228, 0, 2, 318, 476, 8, 2153, 364, 47, 46, 6, 348, 'pingpong.Perhaps', 30, 23, 106, 7, 'foursome', 28, 99, 0], [609, 133, 6, 20, 17, 140, 88, 19, 1180, 1, 30, 121, 250, 99, 6, 47, 1383, 28, 'us.That', 10, 771, 645, 8, 449, 1, 78, 0], ['Good.Let', 161, 34, 47, 92, 0], [219, 55, 0], []]

 [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1,

In [8]:
my_dataset = pre_process.transformed_train_dataset

In [9]:
class chr_rnn(nn.Module):
    def __init__(self, hidden_size, bidirectional):
        super(chr_rnn, self).__init__()
        self.hidden_size = hidden_size
        if(bidirectional == True):
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.chr_rnn = nn.GRU(100, 100, bidirectional = bidirectional)
        
        
    def forward(self,char,h0):
        _,h0 = self.chr_rnn(char,h0)
        return h0
        
    def initHidden(self):
        return torch.zeros(self.bidirectional , 1, self.hidden_size, device=device)
    
    
    
    
class last_net(nn.Module):
    def __init__(self):
        super(last_net, self).__init__()
        self.lastnet = nn.Linear(200,100)
        
    def forward(self,last_hidden_state):
        last_w = self.lastnet(last_hidden_state)
        return last_w
  
    
chr_rnn_1 = chr_rnn(100, True).cuda()
last_net_1 = last_net().cuda()


In [10]:
if os.path.isfile('/home/jongsu/Desktop/persona_dialogue/chr_parameter/my_character_rnn_526.pth'):
    print("loading chr")
    chr_rnn_1.load_state_dict(torch.load('/home/jongsu/Desktop/persona_dialogue/chr_parameter/my_character_rnn_526.pth'))

if os.path.isfile('/home/jongsu/Desktop/persona_dialogue/chr_parameter/my_character_linear_526.pth'):
    print("loading last")
    last_net_1.load_state_dict(torch.load('/home/jongsu/Desktop/persona_dialogue/chr_parameter/my_character_linear_526.pth'))

loading chr
loading last


In [11]:
wv_model_en = word2vec.Word2Vec(size=100, window=5, min_count=5, workers=4)
wv_model_en = word2vec.Word2Vec.load('/home/jongsu/Desktop/persona_dialogue/wv_parameter/dialogue_wv')

In [12]:
class makesent_gru(nn.Module):
    def __init__(self, hidden_size, bidirectional):
        super(makesent_gru, self).__init__()
        self.hidden_size = hidden_size
        if(bidirectional == True):
            self.bidirectional = 2
        else:
            self.bidirectional = 1
        self.gru = nn.GRU(100, 100, bidirectional = bidirectional)
        
        
    def forward(self,char,h0):
        gru,h0 = self.gru(char,h0)
        return h0
        
    def initHidden(self):
        return torch.zeros(self.bidirectional , 1, self.hidden_size, device=device)
 

In [13]:
def log_sum_exp(vec):
    max_score = vec[0, argmax(vec)]
    max_score_broadcast = max_score.view(1, -1).expand(1, vec.size()[1])
    return max_score + \
        torch.log(torch.sum(torch.exp(vec - max_score_broadcast)))
def argmax(vec):
    # return the argmax as a python int
    _, idx = torch.max(vec, 1)
    return idx.item()


def prepare_sequence(seq, to_ix):
    idxs = [to_ix[w] for w in seq]
    return torch.tensor(idxs, dtype=torch.long)


In [14]:
class BiGru_CRF(nn.Module):
    def __init__(self, tagset_set ,hidden_dim):
        super(BiGru_CRF, self).__init__()
        
        self.hidden_dim = hidden_dim
        self.tagset_size = len(tagset_set)

        
        self.gru = nn.GRU(100, 100, bidirectional=True)
        self.gru_postp = nn.Linear(200,100)
        
        self.hidden2tag = nn.Linear(hidden_dim, self.tagset_size+2)
        self.transitions = nn.Parameter(torch.randn(self.tagset_size + 2, self.tagset_size + 2))
        self.hidden = self.init_hidden()
        
        self.transitions.data[0, :] = -10000
        self.transitions.data[:, 29] = -10000
        
    def init_hidden(self):
        return (torch.zeros(2, 1, self.hidden_dim).cuda())

        
    def _forward_alg(self, feats):
        
        # Do the forward algorithm to compute the partition function
        init_alphas = torch.full((1, self.tagset_size+2), -10000.).cuda()
        init_alphas[0][0] = 0.

        forward_var = init_alphas

        # Iterate through the sentence
        for feat in feats:
            alphas_t = []  # The forward tensors at this timestep
            for next_tag in range(self.tagset_size+2):
                # broadcast the emission score: it is the same regardless of
                # the previous tag
                emit_score = feat[next_tag].view(1, -1).expand(1, self.tagset_size+2)
                # the ith entry of trans_score is the score of transitioning to
                # next_tag from i
                trans_score = self.transitions[next_tag].view(1, -1)
                # The ith entry of next_tag_var is the value for the
                # edge (i -> next_tag) before we do log-sum-exp
                
                next_tag_var = forward_var + trans_score + emit_score
                # The forward variable for this tag is log-sum-exp of all the
                # scores.
                alphas_t.append(log_sum_exp(next_tag_var).view(1))
            forward_var = torch.cat(alphas_t).view(1, -1)
            
        terminal_var = forward_var  + self.transitions[29]
        alpha = log_sum_exp(terminal_var)
        return alpha
    
    def _get_gru_features(self, sentence):
        
        self.hidden = self.init_hidden()
        #gru_out, self.hidden = self.gru(sentence, self.hidden)
        #gru_out = self.gru_postp(gru_out)
        gru_out = sentence.view(len(sentence), 100)
        gru_feats = self.hidden2tag(gru_out)
        return gru_feats
    
    def _score_sentence(self, feats, tags):
        # Gives the score of a provided tag sequence
        score = torch.zeros(1).cuda()
        tags = np.append(0,tags)
        
        my_feats = feats
        for i, feat in enumerate(my_feats):
            score = score + \
                self.transitions[tags[i + 1], tags[i]] + feat[tags[i + 1]]
        
        return score
    
    def neg_log_likelihood(self, sentence, tags):
        
        feats = self._get_gru_features(sentence)
        forward_score = self._forward_alg(feats)
        
        gold_score = self._score_sentence(feats, tags)
        return forward_score - gold_score

    
    
    
    def _viterbi_decode(self, feats):
        backpointers = []

        # Initialize the viterbi variables in log space
        init_vvars = torch.full((1, 30), -10000.)
        init_vvars[0][0] = 0

        # forward_var at step i holds the viterbi variables for step i-1
        forward_var = init_vvars
        for feat in feats:
            bptrs_t = []  # holds the backpointers for this step
            viterbivars_t = []  # holds the viterbi variables for this step

            for next_tag in range(self.tagset_size+2):
                
                next_tag_var = forward_var + self.transitions[next_tag]
                best_tag_id = argmax(next_tag_var)
                bptrs_t.append(best_tag_id)
                viterbivars_t.append(next_tag_var[0][best_tag_id].view(1))
            
            forward_var = (torch.cat(viterbivars_t) + feat).view(1, -1)
            backpointers.append(bptrs_t)

        
        terminal_var = forward_var + self.transitions[29]
        best_tag_id = argmax(terminal_var)
        path_score = terminal_var[0][best_tag_id]

        
        best_path = [best_tag_id]
        for bptrs_t in reversed(backpointers):
            best_tag_id = bptrs_t[best_tag_id]
            best_path.append(best_tag_id)
        # Pop off the start tag (we dont want to return that to the caller)
        start = best_path.pop()
        assert start == 0  # Sanity check
        best_path.reverse()
        return path_score, best_path
    
    
    
    
    def forward(self, sentence):  # dont confuse this with _forward_alg above.
        # Get the emission scores from the BiLSTM
        lstm_feats = self._get_gru_features(sentence)

        # Find the best path, given the features.
        score, tag_seq = self._viterbi_decode(lstm_feats)
        return score, tag_seq

In [15]:
class last_sent_net(nn.Module):
    def __init__(self):
        super(last_sent_net, self).__init__()
        self.lastnet = nn.Linear(200,100)
        
    def forward(self,last_hidden_state):
        last_w = self.lastnet(last_hidden_state)
        return last_w


In [16]:
tagset = torch.tensor(np.arange(28),device=device)

In [17]:
sentence_1 = makesent_gru(100,True).cuda()
last_sent_net1 = last_sent_net().cuda()
Bigru_crf1 = BiGru_CRF(tagset,100).cuda()

In [18]:

if os.path.isfile('/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/crf1.pth'):
    print("loading crf")
    Bigru_crf1.load_state_dict(torch.load('/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/crf1.pth'))

if os.path.isfile('/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/crf2.pth'):
    print("loading last")
    sentence_1.load_state_dict(torch.load('/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/crf2.pth'))

if os.path.isfile('/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/crf3.pth'):
    print("loading last")
    last_sent_net1.load_state_dict(torch.load('/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/crf3.pth'))
    


loading crf
loading last
loading last


In [19]:
optimizer1 = optim.SGD(Bigru_crf1.parameters(), lr=0.0001, weight_decay=1e-4)
optimizer2 = optim.SGD(sentence_1.parameters(), lr=0.0001, weight_decay=1e-4)
optimizer3 = optim.SGD(last_sent_net1.parameters(), lr=0.0001, weight_decay=1e-4)

epoch = 0

while(epoch < 100): 
    
    total_lss = 0
    dialogue_idx = 0
    print("new iter")
    while(dialogue_idx < len(my_dataset)):
        
        #print("dialogue_idx",dialogue_idx)
        per_dialogue = my_dataset[dialogue_idx]
        sent_idx = 0
        new_sent_to_vec = []
        sent_to_vec = torch.tensor(new_sent_to_vec).cuda()
        
        make_label = []
        while(sent_idx < len(per_dialogue['dialogue'][0]) -1): #indexed dialogue
            
            
            
            ## sent_lstm training
            sent = per_dialogue['dialogue'][0][sent_idx]
            sent_len = len(per_dialogue['dialogue'][0][sent_idx]) 
            
            word_idx = 0
            vec = []
            
            while(word_idx < sent_len): 
                
                
                encoder_hidden = chr_rnn_1.initHidden()
                if per_dialogue['dialogue'][1][sent_idx][word_idx] == 1: #word_to_vec
                    word_to_tensor = torch.tensor(wv_model_en.wv[wv_model_en.wv.index2entity[sent[word_idx]]])
                    gpu_wt = word_to_tensor.cuda()
                    vec.append(gpu_wt) #append known word vec
                else: #character model
                    for ei in range(len(sent[word_idx])):
                        gru_input = torch.tensor(np.zeros(shape = (1, 1, 100), dtype="float32")).cuda()
                        gru_input[0][0][ord(sent[word_idx][ei])%100] = 1 #character to onehot vec
                        encoder_hidden = chr_rnn_1(gru_input,encoder_hidden)

                    last_encoder_hidden = encoder_hidden.view(-1,200)
                    last_encoder_hidden_ = last_net_1(last_encoder_hidden) #last hiddenstate and fully c l
                    vec.append(last_encoder_hidden_) #append unknown word vec
                word_idx = word_idx + 1
            
            
            word_idx = 0 
            
            sent_hidden = sentence_1.initHidden()
            
            
            while(word_idx<sent_len):

                view = vec[word_idx].view(1,1,100)
                sent_hidden = sentence_1(view,sent_hidden)
                word_idx = word_idx + 1
            
            # 200 -> 100
            
            sent_vec = last_sent_net1(sent_hidden.view(-1,200))
            sent_to_vec = torch.cat((sent_to_vec,sent_vec),0)
            
            try:
                index1_toint = int(per_dialogue['label'][0][sent_idx*2]) *4
                index2_toint = int(per_dialogue['label'][1][sent_idx*2])
                make_label.append(index1_toint + index2_toint)
            except:
                print("data_err")
                make_label.append(0)
            
            sent_idx = sent_idx + 1
        

        resh_stv = sent_to_vec.view(-1,1,100)
        torch_make_label = torch.tensor(make_label).cuda()
        loss = Bigru_crf1.neg_log_likelihood(resh_stv, make_label)
        ## lstm_crf training
        
        dialogue_idx = dialogue_idx + 1
        
        total_lss = total_lss + loss
        
        if(dialogue_idx % 100 ==0):
            print("cal grad..")
            Bigru_crf1.zero_grad()
            sentence_1.zero_grad()
            last_sent_net1.zero_grad()
            
            total_lss.backward()
            print(total_lss)
            
            optimizer1.step()
            optimizer2.step()
            optimizer3.step()
            
            total_lss = 0
            
            torch.save(Bigru_crf1.state_dict(),'/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/crf1_final.ckpt')
            torch.save(sentence_1.state_dict(),'/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/crf2_final.ckpt')
            torch.save(last_sent_net1.state_dict(),'/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/crf3_final.ckpt')

            torch.save(Bigru_crf1.state_dict(),'/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/crf1_final.pkl')
            torch.save(sentence_1.state_dict(),'/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/crf2_final.pkl')
            torch.save(last_sent_net1.state_dict(),'/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/crf3_final.pkl')

            torch.save(Bigru_crf1.state_dict(),'/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/crf1_final.pth')
            torch.save(sentence_1.state_dict(),'/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/crf2_final.pth')
            torch.save(last_sent_net1.state_dict(),'/home/jongsu/Desktop/persona_dialogue/crf_lstm_parameter/crf3_final.pth')
        
    epoch = epoch + 1

new iter
cal grad..
tensor([ 156.0618], device='cuda:0')
cal grad..
tensor([ 137.1423], device='cuda:0')
cal grad..
tensor([ 126.0625], device='cuda:0')
cal grad..
tensor([ 138.1004], device='cuda:0')
cal grad..
tensor([ 106.7880], device='cuda:0')
cal grad..
tensor([-18.4861], device='cuda:0')
cal grad..
tensor([ 51.9405], device='cuda:0')
cal grad..
tensor([ 191.9272], device='cuda:0')
cal grad..
tensor([ 191.6957], device='cuda:0')
cal grad..
tensor([ 334.3360], device='cuda:0')
cal grad..
tensor([ 173.1140], device='cuda:0')
cal grad..
tensor([ 26.5188], device='cuda:0')
cal grad..
tensor([ 296.8061], device='cuda:0')
cal grad..
tensor([ 91.6075], device='cuda:0')
cal grad..
tensor([ 181.0750], device='cuda:0')
cal grad..
tensor([ 257.3882], device='cuda:0')
cal grad..
tensor([ 327.5539], device='cuda:0')
cal grad..
tensor([ 196.1014], device='cuda:0')
cal grad..
tensor([-28.6332], device='cuda:0')
cal grad..
tensor([ 144.9202], device='cuda:0')
cal grad..
tensor([ 26.6877], device

KeyboardInterrupt: 